参考[网站](https://blog.csdn.net/qq_47233366/article/details/122611672)  
用于熟悉流程，完整代码参见demo.py

## 0. 导入包

In [1]:
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## 1. 准备数据集

In [2]:
train_dataset = torchvision.datasets.CIFAR10(
    './data',    # 指定下载数据集保存的位置
    train=True,   # 指下载的数据是训练集数据还是测试集数据【True表示训练集，Flase表示测试集】
    transform=torchvision.transforms.ToTensor(),   # 图片的一个转化，要将图片格式转化为tensor类型
    download=True   # download为True表示你没有这个数据，这时候会自动下载数据，为Flase表示有这个数据，不会再进行下载【注意：这个参数设置成True且你有数据集，那同样不会进行数据下载，故这个参数一直设置成True就好了】
)
test_dataset = torchvision.datasets.CIFAR10(
    './data',
    train=False,
    transform=torchvision.transforms.ToTensor(),
    download=True
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 打印数据集的长度来看一下这个数据集的大小
train_dataset_size = len(train_dataset)
test_dataset_size = len(test_dataset)
print('train_dataset_size:{}'.format(train_dataset_size))
print('test_dataset_size:{}'.format(test_dataset_size))

train_dataset_size:50000
test_dataset_size:10000


## 2. 加载数据集

In [4]:
train_dataset_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64
)
test_dataset_loader = DataLoader(
    dataset=test_dataset,
    batch_size=64
)

![](https://raw.githubusercontent.com/XM-Chen/figuremap/main/my_notes/202310311642928.png)

## 3. 搭建神经网络

![](https://raw.githubusercontent.com/XM-Chen/figuremap/main/my_notes/202310311644101.png)  
根据上图来搭建网络模型

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model1 = nn.Sequential(
            nn.Conv2d(3, 32, 5, padding=2),   # 第一个卷积层，输入通道为3，输出通道为32，卷积核大小为5*5
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10),
        )
    
    
    def forward(self, input):
        input = self.model1(input)
        return input

## 4. 创建网络模型

In [6]:
net = Net().to(device)
print(net)

Net(
  (model1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=1024, out_features=64, bias=True)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)


 到这里我们已经创建好了自己的模型，这个模型输入是3x32x32的图片【可以认为就是一个3x32x32的张量】，输出是1x10的向量。
 
 每当我们创建好一个模型后，应该检测一下模型的输入输出是否是我们所期待的，若不是则及时调整模型。我们可以用以下代码来检测输出是否符合要求。

In [7]:
# net = Net()
# input = torch.randn([64, 3, 32, 32])  # 64是输入的batch_size
# output = net(input)
# print(output.size())

## 5. 设置损失函数，优化器

In [8]:
# 损失函数
loss_func = nn.CrossEntropyLoss()
loss_func = loss_func.to(device)


# 优化器
learn_rate = 0.001
optimizer = torch.optim.Adam(net.parameters(), lr=learn_rate)

## 6. 设置网络训练中的一些参数
这部分主要是用来记录一些训练测试的次数及网络训练轮数。

In [9]:
total_train_step = 0   # 记录总计训练的次数
total_test_step = 0    # 记录总计测试的次数
epoch = 20              # 设计训练的轮数

## 7. 开始训练网络
进行网络训练时，我们首先会通过自己构建的网络得到输出，然后比较输出和真实值，计算出损失，最后通过反向传播，调整网络中参数的值。

In [10]:
for i in range(epoch):
    print('---第{}轮训练开始---'.format(i+1))
    
    net.train()      #开始训练，不是必须的，在网络中有BN，dropout时需要
    for data in train_dataset_loader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        output = net(imgs)
        
        # 比较输出与真实值，计算Loss
        loss = loss_func(output, targets)
        
        # 反向传播,调整参数
        optimizer.zero_grad()  # 每次让梯度重置
        loss.backward()
        optimizer.step()   # 用于更新模型的参数
        
        total_train_step += 1
        
        if total_train_step % 100 == 0:
            print('---第{}次训练结束，Loss={}'.format(total_train_step, loss.item()))
            

---第1轮训练开始---
---第100次训练结束，Loss=1.7066161632537842
---第200次训练结束，Loss=1.7024245262145996
---第300次训练结束，Loss=1.4710195064544678
---第400次训练结束，Loss=1.2554702758789062
---第500次训练结束，Loss=1.2134944200515747
---第600次训练结束，Loss=1.3010448217391968
---第700次训练结束，Loss=1.3992218971252441
---第2轮训练开始---
---第800次训练结束，Loss=1.0933609008789062
---第900次训练结束，Loss=1.059903860092163
---第1000次训练结束，Loss=1.2663791179656982
---第1100次训练结束，Loss=1.2637253999710083
---第1200次训练结束，Loss=1.0375734567642212
---第1300次训练结束，Loss=1.0435911417007446
---第1400次训练结束，Loss=0.8742586374282837
---第1500次训练结束，Loss=1.1345350742340088
---第3轮训练开始---
---第1600次训练结束，Loss=0.7971119284629822
---第1700次训练结束，Loss=0.8544291257858276
---第1800次训练结束，Loss=0.9309648871421814
---第1900次训练结束，Loss=1.0064424276351929
---第2000次训练结束，Loss=1.2302665710449219
---第2100次训练结束，Loss=0.7614014744758606
---第2200次训练结束，Loss=0.6597464084625244
---第2300次训练结束，Loss=1.1109529733657837
---第4轮训练开始---
---第2400次训练结束，Loss=0.986819326877594
---第2500次训练结束，Loss=0.8223930597305298
---第2

## 8. 开始测试网络
对网络进行测试过程和训练是类似的，不同的是测试过程不需要通过反向传播来更新参数。

In [11]:
net.eval()   #开始测试，不是必须的，在网络中有BN，dropout时需要
with torch.no_grad():   #这句表示测试不需要进行反向传播，即不需要梯度变化【可以不加】
    total_test_loss = 0    # 测试损失
    total_test_accuracy = 0  # 测试集准确率
    for data in test_dataset_loader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = net(imgs)
        
        # 计算测试损失
        loss = loss_func(outputs, targets)
        total_test_loss = total_test_loss + loss.item()
        accuracy = (outputs.argmax(1) == targets).sum()
        total_test_accuracy = total_test_accuracy + accuracy
print('第{}轮测试的总损失为：{}'.format(i+1, total_test_loss))
print('第{}轮测试的准确率为：{}'.format(i+1, total_test_accuracy/test_dataset_size))

第20轮测试的总损失为：350.32522106170654
第20轮测试的准确率为：0.6243999600410461


## 保存模型

In [12]:
torch.save(net, './self_model_{}.pth'.format(i+1))
print('模型已保存')

模型已保存
